In [160]:
import pandas as pd
import os
import requests

In [164]:
raidDir   = '/Users/amcguire/Documents/PERSONAL/KOL/raidLogs/'
#raidDir   = '/Users/amcguire/Documents/PERSONAL/KOL/raidLogsPreJune6/'
newRDir   = '/Users/amcguire/Library/Application Support/KoLmafia/data/test/'
outputDir = '/Users/amcguire/Documents/PERSONAL/KOL/'

# users who are running dread w/ alts
multiUser = {'madowl (combined)':['themadowl','madowl','Arkcon']}

# clan maps. keeping HoD in here even tho i took it out of my clanHop.ash
clanMap   = {'The Hogs of Exploitery':'HoE', 
             'KirbyLlama\'s Private Dungeon Clan': 'KirbyClan',
             'The Piglets of Fate': 'Piglets',
             'The Hogs of Destiny': 'HoD',
             'The 100% of Scientific FACT Club': 'FactClub',
             'Castle Greyhawk': 'Greyhawk',
             'I have no idea what\'s going on':'NoIdea'}

# folks who have all their skills and thus are outta the skill ladder
finishedUsers = ['violetinsane','Captain Scotch','kenny kamAKAzi','The Dictator',
                 'madowl','gregmasta','RandomExtremity']

# run files from my old EXTREMELY manual process
runFiles  = ['r1-3389','r2-3869','r3-4283','r4-4190','r5-3503','r6-3598',
             'r7-4372','r8-3473','r9-3858','r10-3787','r11-3797','r12-3798',
             'r13-12260','r14-8393','r15-3769','r16-3496','r17-4041','r18-3582',
             'r19-4318','r20-3584','r21-3794','r22-4698','r25-3802','r69-1337','r189353_KirbyClan']

runFilesPhil = ['r189472_HoE','r189711_Greyhawk','r188769_HoE',
                'r188757_KirbyClan','r189759_HoE','r188819_HoD','r189658_FactClub',
                'r189559_Greyhawk','r189659_KirbyClan','r188871_Greyhawk','r188579_Piglets',
                'r189896_KirbyClan','r189712_KirbyClan','r188572_HoD','r189560_HoE',
                'r189481_Piglets','r189236_HoE','r189900_FactClub','r189252_Piglets',
                'r189733_Piglets','r189268_KirbyClan','r189482_FactClub','r189332_FactClub',
                'r188955_KirbyClan','r189415_HoD','r189861_Greyhawk','r189010_HoD','r189082_HoE',
                'r189434_KirbyClan','r188926_Piglets','r189409_KirbyClan','r189169_KirbyClan',
                'r189327_HoE','r189553_KirbyClan','r189197_Piglets','r189829_Piglets',
                'r189544_Piglets']

In [165]:
# # Temporary script to convert Stary script output into old c/p format

for file in os.listdir(newRDir): # i wish listdir could give datetime info
    print(file)
    try:    
        clan = file.split(sep='_')[1]
        rNum = file.split(sep='_')[2][:6]
        newFileName = 'r{1}_{0}.txt'.format(clanMap[clan],rNum)

        currFile = open('{}/{}'.format(newRDir, file)).read()

        newFile = ''

        for ct, line in enumerate(currFile.split(sep='\n')):
            try:     newFile = newFile + line.split(sep='\t')[1].replace('<b>','').replace('</b>','') + '\n'
            except:  newFile = newFile + 'LINE-ERROR' + '\n'

        print('  --- > {} file parsed. {} new lines.'.format(clan,ct))

        # it is kind of dumb that i just keep writing to my old raidLog directory 
        #   in old raidLog format but i mean, the code works, so why mess it up
        #   too much i guess ???
        file = open('{}{}'.format(raidDir,newFileName),"w")

        file.write(newFile)
        file.close()

        runFiles = runFiles + [newFileName[:-4]]
    except:
        print('  --- !!!!!!!!!!!!!!!! stupid thing fucked up !!!!!!!!!!!!!!!!')

dungeon_Castle Greyhawk_190069.txt
  --- > Castle Greyhawk file parsed. 221 new lines.
dungeon_KirbyLlama's Private Dungeon Clan_190027.txt
  --- > KirbyLlama's Private Dungeon Clan file parsed. 181 new lines.
dungeon_The Hogs of Exploitery_189472.txt
  --- > The Hogs of Exploitery file parsed. 115 new lines.
dungeon_Castle Greyhawk_189711.txt
  --- > Castle Greyhawk file parsed. 187 new lines.
.DS_Store
  --- !!!!!!!!!!!!!!!! stupid thing fucked up !!!!!!!!!!!!!!!!
dungeon_The Hogs of Exploitery_189927.txt
  --- > The Hogs of Exploitery file parsed. 186 new lines.
dungeon_The Hogs of Exploitery_188769.txt
  --- > The Hogs of Exploitery file parsed. 184 new lines.
dungeon_KirbyLlama's Private Dungeon Clan_188757.txt
  --- > KirbyLlama's Private Dungeon Clan file parsed. 189 new lines.
dungeon_The 100% of Scientific FACT Club_190068.txt
  --- > The 100% of Scientific FACT Club file parsed. 231 new lines.
dungeon_The Hogs of Exploitery_189759.txt
  --- > The Hogs of Exploitery file parse

In [166]:
# Populate our run dictionary

# From helping set up OAF
#runFiles = runFiles + runFilesPhil

allRuns = ""

for run, txtLog in enumerate(runFiles):
    # Python is zero-indexed, so we increment run by 1 here. 
    #   Also replace spaces w/ _ for playername.
    
    allRuns = allRuns+open('{}{}.txt'.format(raidDir, txtLog)).read()+"\n"

In [167]:
def collectRaiders(runs):
    """ Function to collect all dungeon raiders. Only grabs folks who
        have defeated at least one monster across all ASS runs. """
    
    raiders = []
    
    for row in runs.split('\n'):
        if ('defeated' in row) & (' (#' in row):
            name = row[:row.find(' (#')]
            raiders = list(set(raiders + [name]))
    
    return raiders
  
# ... why did I make this function, anyway?
# collectRaiders(allRuns)

In [168]:
def tabulateKills(name, runs):
    """ Function to tabulate how many kills a specific individual had.
        Also outputs defeats, if we actually want that. """
    
    kills  = 0
    killed = 0
    
    for row in runs.split('\n'):
        if (name in row) & ('defeated' in row):
            if row[0:len(name)] == name: # fixing the madowl bug ;-;
                if 'was defeated' in row:
                    tSpent = int(row.split(' (')[2].split(' ')[0])
                    killed += tSpent
                else:
                    tSpent = int(row.split(' (')[2].split(' ')[0])
                    kills  += tSpent
                
    return({'kills':kills,'defeats':killed})

In [169]:
def tabulateLoots(name, runs):
    ''' Function to tabulate loot per individuals participating. Also 
        grabs skills, just so that we have em.'''
    
    loot   = []
    skills = 0
    
    for row in runs.split('\n'):
        if ('distributed' in row) and (name in row):
            distTo = row.split(' to ')[1].split(' (#')[0]
            if distTo == name:
                thisLoot = row.split(' distributed ')[1].split(' to ')[0]
                loot = loot + [thisLoot]
        elif (' used The Machine' in row) and (name in row):
            distTo = row.split(' used The ')[0].split(' (#')[0]
            if distTo == name: skills+=1
        
    # Parse out things we don't want to be considered as "true" loot
    notRealLoot = ['ghost pepper','electric Kool-Aid','skull capacitor',
                   'wriggling severed nose','Hunger™ Sauce','bottle of Bloodweiser']
    finalLoot = [x for x in loot if x not in notRealLoot]
    
    return({'loot':finalLoot,'skills':skills})

# unfortunately stary's clanHop script doesn't actually save loot. so, basically all loot is sourced from my manual logs  
# tabulateLoots('Captain Scotch', allRuns)

In [170]:
finalDisplay = {}

# Create summary dictionary via tabulation code

# ... oh, THAT'S why i made collectRaiders, yes, this makes sense
for name in collectRaiders(allRuns):
    
    finalDisplay[name] = {}
    
    finalDisplay[name].update(tabulateKills(name, allRuns))
    finalDisplay[name].update(tabulateLoots(name, allRuns))
    finalDisplay[name]['finishedFlag'] = [1 if name in finishedUsers else 0][0]
    
    finalDisplay[name]['lootCount'] = len(finalDisplay[name]['loot'])

In [171]:
# Slice of code for combining duplicate users

for name in multiUser.keys():
    
    finalDisplay[name] = {}
    
    for field in ['kills','defeats','loot','skills','lootCount','finishedFlag']:
        try:    finalDisplay[name][field] = sum([finalDisplay[x][field] for x in multiUser[name]])
        except: finalDisplay[name][field] = [].append([finalDisplay[x][field] for x in multiUser[name]])
            
    for x in multiUser[name]: finalDisplay.pop(x)

In [172]:
finalFrame = pd.DataFrame.from_dict(finalDisplay,orient='index').sort_values(by='kills',ascending=False).fillna(0)
finalFrame = finalFrame.loc[:,['kills','defeats','skills','lootCount','loot','finishedFlag']]
finalFrame['kill/skillz'] = finalFrame['kills']/(finalFrame['skills']+0.5)

finalFrame = finalFrame.sort_values(['finishedFlag','kill/skillz'], ascending=[True,False])
finalFrame

,kills,defeats,skills,lootCount,loot,finishedFlag,kill/skillz
Phillammon,7396,78,6,15,"[Drunkula's bell, Gets-You-Drunk, HOA regulati...",0,1137.846154
Liora,548,8,0,1,[Great Wolf's left paw],0,1096.000000
Name Guy Man,9844,7,9,17,"[Quiets-Your-Steps, Covers-Your-Head, Drunkula...",0,1036.210526
Stary,10649,92,10,7,"[Mayor Ghost's scissors, Drunkula's cape, Grea...",0,1014.190476
Whym,5368,1,5,1,[Gets-You-Drunk],0,976.000000
Volotani,8279,64,8,12,"[Mayor Ghost's khakis, Mayor Ghost's scissors,...",0,974.000000
threebullethamburgler,10132,166,10,11,"[Drunkula's silky pants, Mayor Ghost's cloak, ...",0,964.952381
chuggy,2342,0,2,0,[],0,936.800000
stibarsen,8868,17,9,0,[],0,933.473684
chameco,4197,0,4,1,[Gets-You-Drunk],0,932.666667


In [173]:
print("This effort has gotten {} skills for {} Loathers. Good job folks!".format(finalFrame['skills'].sum(),len(finalFrame['skills'])))

This effort has gotten 188 skills for 34 Loathers. Good job folks!


In [174]:
from datetime import date

currDate = str(date.today().strftime('%m%d'))
finalFrame.to_csv('{}dungeonLogs{}.csv'.format(outputDir,currDate))

In [175]:
def lineParse(string):
    ''' The buffer bit is when I was erroneously thinking that defeats somehow 
        counted against us. They do not. Still, left the logic in just in case
        I want to make this more complicated in the future, and get differential
        information for different parsing situations.'''
    
    if 'was defeated' in string:
        return {'buffer':int(string.split(' (')[2].split(' ')[0]),'kills':0}
    else:
        try:
            if ') defeated' not in string: print(string)
            return {'kills':int(string.split(' (')[2].split(' ')[0]),'buffer':0}
        except: 
            # print out when this encounters an error; this is how i found all the exceptions below
            print(string)
            return {'buffer':0,'kills':0}

def tabulateSummary(runs, printFlag = True):
    """ Function to tabulate a general summary of a run. """
    
    # i am 100000% positive there is a better way to do this, or to mass-assign
    #   null variables. i cannot for the life of me remember what it is.
    
    woodBossKilled = 0
    villBossKilled = 0
    castBossKilled = 0
    
    bugbear  = 0
    werewolf = 0
    
    zombie   = 0
    ghost    = 0
    
    skeleton = 0
    vampire  = 0
    
    machine = 0
    
    skullcap = 0
    
    # some loot/actions mess w/ the string parsing here; manually exclude em
    exclTable = ['ghost pepper', 'HOA zombie eyes', 'zombie accordion', 
                 'zombie mariachi pants', 'zombie mariachi hat', 
                 'drove some zombies out', 'ghost pencil', 'ghost shawl',
                 'drove some skeletons out', 'drove some vampires']
    
    # this isn't particularly elegant but it's fast
    for row in runs.split('\n'):
        if any(excl in row for excl in exclTable):
            continue
        elif ('bugbear'  in row):
            bugbear  += lineParse(row)['kills']
        elif ('werewolf' in row):
            werewolf  += lineParse(row)['kills']
        elif ('zombie'   in row):
            zombie  += lineParse(row)['kills']
        elif ('ghost'    in row):
            ghost  += lineParse(row)['kills']
        elif ('skeleton' in row):
            skeleton  += lineParse(row)['kills']
        elif ('vampire'  in row):
            vampire  += lineParse(row)['kills']
        elif ('used The Machine' in row):
            machine += 1
        elif ('fixed The Machine' in row):
            skullcap += 1
            
        # bosses; this is currently unnecessary but i was curious how many 
        #   bosskillings i actually captured, since i'm not the only one who
        #   cycles instances and i don't always snag an update beforehand
        
        elif ('  Falls-From-Sky' in row):
            woodBossKilled += lineParse(row)['kills']
        elif ('The Great Wolf of the Air ' in row):
            woodBossKilled += lineParse(row)['kills']
        elif ('  Mayor Ghost' in row):
            villBossKilled += lineParse(row)['kills']
        elif (' the Zombie Homeowners\' Association ' in row):
            villBossKilled += lineParse(row)['kills']
        elif ('  Count Drunkula ' in row):
            castBossKilled += lineParse(row)['kills']
        elif ('  The Unkillable Skeleton' in row):
            castBossKilled += lineParse(row)['kills']
    
    # was bugchecking a stupid error
    # print("{} {} | {} {} | {} {}".format(bugbear,werewolf,zombie,ghost,vampire,skeleton))
    
    woodPct = bugbear/(bugbear+werewolf+0.00001)
    villPct = zombie/(zombie+ghost+0.00001)
    castPct = vampire/(vampire+skeleton+0.00001)
    
    # my initial thought was that anything over 50% guaranteed that boss; this is 
    #   not strictly true apparently, and you can occasionally get flips w/ <60%. 
    #   so this checks so that we have a clear sense of what the exact diff is
    
    if woodPct>0.5: woodBoss = 'FFS ({:0.2f})'.format(woodPct)      # there is a % format but yolo
    else:           woodBoss = 'Wolf ({:0.2f})'.format(1-woodPct)
        
    if villPct>0.5: villBoss = 'ZHoA ({:0.2f})'.format(villPct)
    else:           villBoss = 'MG ({:0.2f})'.format(1-villPct)
        
    if castPct>0.5: castBoss = 'Drunkula ({:0.2f})'.format(castPct)
    else:           castBoss = 'UKS ({:0.2f})'.format(1-castPct)
    
    # check if a cap has been placed
    if skullcap > 0:
        capStatus = ':phillsgood:'
    else:
        capStatus = ':phillsbad:'
    
    # (the stupid error was that the castle was "skeleton - zombie" so i had a bunch of weird results)
    if printFlag == True:   
        print('==== RUN SUMMARY ==== \n   KILLS LEFT: {}/{}/{}\n   BOSSES: {}, {}, {}\n   MACHINE: {} skills left!\n   BOSS DEFEATS: {} {} {}'.format(
        1000-(bugbear+werewolf),1000-(ghost+zombie),1000-(skeleton+vampire),woodBoss,villBoss,castBoss,3-machine,woodBossKilled, villBossKilled, castBossKilled))
    
    if capStatus == ':phillsgood:':
        # return is for the text update creator and naught else
        return '{}/{}/{} ({} skills left! -- {}, {}, {})'.format(
            1000-(bugbear+werewolf),1000-(ghost+zombie),1000-(skeleton+vampire),3-machine,woodBoss,villBoss,castBoss)
    
    elif capStatus == ':phillsbad:':
        # return is for the text update creator and naught else
        return '{}/{}/{} (no capacitor yet! {})'.format(
            1000-(bugbear+werewolf),1000-(ghost+zombie),1000-(skeleton+vampire),capStatus)


In [176]:
# commented out because it's quite long. but if you want to summarize everything, this is it!

# for run in runFiles:
#     print('=====================\n---- {}'.format(run))
#     tabulateSummary(open('{}{}.txt'.format(raidDir, run)).read()+"\n")

# also, while i'm commenting stuff out, this is a snippet to get a player's involvement in all runs

# n = "The Dictator"
# for run in runFiles:
#     print('{} - {}'.format(run,str(tabulateKills(n, open('{}{}.txt'.format(raidDir, run)).read()+"\n"))))

In [177]:
def buildTextUpdate(numInst = 5, rDir = newRDir,desc = 'yolo yolo yolo'):
    ''' Grabs the five most recent instances & generates info '''
    
    # most recent files using some garbo processing. there is inevitably a better
    #   way to access/read this, but i am on a plane with broken wifi, and this 
    #   does technically work on unix systems!
    fileTree = os.popen('ls -lrst "{}"'.format(rDir)).read().split('\n')[-1*(numInst+1):]
    
    infoDump = {}
    orderedL = []
    
    for fileRaw in fileTree:
        # unix system generates this in fixed width; the "53" buffer here is system-dependent
        if len(fileRaw) > 53: # have to have this to error-catch widowed newlines
            file = fileRaw[53:]
            clan = clanMap[file.split(sep='_')[1]]
            
            # get a summary for this specific recent clan
            infoDump[clan] = tabulateSummary(open('{}{}'.format(rDir, file)).read()+"\n",printFlag=False)
    
    # making an ordered list of ordered tuples w/ total kills per clan to order the update 
    for clan in infoDump:
        kLeft = sum([int(kills) for kills in infoDump[clan].split(' ')[0].split('/')])
        orderedL = orderedL + [(kLeft,clan)]
    
    # sort is an inplace operator in python
    orderedL.sort()
    
    clanUpdate = ''
    
    # instead of reversing the sort order i just append on the front. is that dumb? yeah. probably.
    for x in orderedL:
        
        clanUpdate = '**{}:** '.format(x[1]) + infoDump[x[1]] + '\n' + clanUpdate
    
    # print that dang update buddy
    print('** {} ASS DREAD UPDATE **\n\nhey @Dungeon Runners ! {}\n\n{}\nand your skill ladder!\n'.format(currDate,desc,clanUpdate))
    
    # return that dict i built just in case i use it for something else later i guess
    return infoDump

In [178]:
# just update the description and it'll generate a paste-able update! whoo!

description = "here is your update! get your skills."

buildTextUpdate(5,newRDir, desc = description)

179	dread_db (#3317385) drove some bugbears out of the forest (1 turn)
** 0630 ASS DREAD UPDATE **

hey @Dungeon Runners ! here is your update! get your skills.

**KirbyClan:** 10/72/10 (3 skills left! -- Wolf (0.61), ZHoA (0.59), Drunkula (0.60))
**NoIdea:** 10/10/10 (1 skills left! -- Wolf (0.60), MG (0.63), UKS (0.58))
**Greyhawk:** 10/10/10 (3 skills left! -- Wolf (0.58), MG (0.58), UKS (0.61))
**FactClub:** 10/10/10 (0 skills left! -- FFS (0.59), MG (0.57), UKS (0.63))
**HoE:** 9/10/10 (3 skills left! -- Wolf (0.58), ZHoA (0.63), Drunkula (0.62))

and your skill ladder!



{'HoE': '9/10/10 (3 skills left! -- Wolf (0.58), ZHoA (0.63), Drunkula (0.62))',
 'KirbyClan': '10/72/10 (3 skills left! -- Wolf (0.61), ZHoA (0.59), Drunkula (0.60))',
 'NoIdea': '10/10/10 (1 skills left! -- Wolf (0.60), MG (0.63), UKS (0.58))',
 'Greyhawk': '10/10/10 (3 skills left! -- Wolf (0.58), MG (0.58), UKS (0.61))',
 'FactClub': '10/10/10 (0 skills left! -- FFS (0.59), MG (0.57), UKS (0.63))'}

In [179]:

allRuns = ""

for run, txtLog in enumerate(runFiles):
    # Python is zero-indexed, so we increment run by 1 here. 
    #   Also replace spaces w/ _ for playername.
    print('{} added at {}'.format(txtLog, allRuns.count('\n')))
    allRuns = allRuns+open('{}{}.txt'.format(raidDir, txtLog)).read()+"\n"
    
x = 0
for line in allRuns.split('\n'):
    x = x + 1
    if "Gausie (#1197090) used The Machine, assisted by Just Eyes and Captain Scotch" in line:
        print('\nThis manual skill was added at line {}.'.format(x))

r1-3389 added at 0
r2-3869 added at 214
r3-4283 added at 507
r4-4190 added at 785
r5-3503 added at 1042
r6-3598 added at 1315
r7-4372 added at 1532
r8-3473 added at 1800
r9-3858 added at 2061
r10-3787 added at 2317
r11-3797 added at 2571
r12-3798 added at 2830
r13-12260 added at 3067
r14-8393 added at 3229
r15-3769 added at 3404
r16-3496 added at 3626
r17-4041 added at 3836
r18-3582 added at 4063
r19-4318 added at 4327
r20-3584 added at 4562
r21-3794 added at 4808
r22-4698 added at 5032
r25-3802 added at 5340
r69-1337 added at 5616
r189353_KirbyClan added at 5806
r190069_Greyhawk added at 5912
r190027_KirbyClan added at 6135
r189472_HoE added at 6318
r189711_Greyhawk added at 6435
r189927_HoE added at 6624
r188769_HoE added at 6812
r188757_KirbyClan added at 6998
r190068_FactClub added at 7189
r189759_HoE added at 7422
r188819_HoD added at 7654
r189658_FactClub added at 7845
r189559_Greyhawk added at 8032
r190187_Greyhawk added at 8168
r189659_KirbyClan added at 8319
r188871_Greyhawk a

In [182]:

# also, while i'm commenting stuff out, this is a snippet to get a player's involvement in all runs

n = "Phillammon"
for run in runFiles:
    print('{} - {}'.format(run,str(tabulateKills(n, open('{}{}.txt'.format(raidDir, run)).read()+"\n"))))

r1-3389 - {'kills': 105, 'defeats': 2}
r2-3869 - {'kills': 676, 'defeats': 16}
r3-4283 - {'kills': 218, 'defeats': 1}
r4-4190 - {'kills': 0, 'defeats': 0}
r5-3503 - {'kills': 0, 'defeats': 0}
r6-3598 - {'kills': 0, 'defeats': 0}
r7-4372 - {'kills': 112, 'defeats': 4}
r8-3473 - {'kills': 15, 'defeats': 1}
r9-3858 - {'kills': 28, 'defeats': 8}
r10-3787 - {'kills': 102, 'defeats': 19}
r11-3797 - {'kills': 1, 'defeats': 0}
r12-3798 - {'kills': 407, 'defeats': 0}
r13-12260 - {'kills': 0, 'defeats': 0}
r14-8393 - {'kills': 466, 'defeats': 0}
r15-3769 - {'kills': 312, 'defeats': 2}
r16-3496 - {'kills': 734, 'defeats': 0}
r17-4041 - {'kills': 328, 'defeats': 0}
r18-3582 - {'kills': 0, 'defeats': 0}
r19-4318 - {'kills': 0, 'defeats': 0}
r20-3584 - {'kills': 0, 'defeats': 0}
r21-3794 - {'kills': 0, 'defeats': 0}
r22-4698 - {'kills': 0, 'defeats': 0}
r25-3802 - {'kills': 0, 'defeats': 0}
r69-1337 - {'kills': 0, 'defeats': 0}
r189353_KirbyClan - {'kills': 0, 'defeats': 0}
r190069_Greyhawk - {'kill